In [ ]:
#!git clone https://github.com/justinpinkney/stable-diffusion.git
%cd 'stable-diffusion'

In [ ]:
# take care!!!!
#!pip install --upgrade pip
#!pip install huggingface_hub
#!pip install -r requirements.txt

#!pip install --upgrade keras
#!pip uninstall -y torchtext

In [ ]:
from PIL import Image
from datasets import load_dataset
from huggingface_hub import notebook_login
from huggingface_hub import hf_hub_download

In [ ]:
!nvidia-smi

In [ ]:
img_dir = 'fashion-dataset/new-images/train'
dataset = load_dataset('imagefolder', data_dir=img_dir,  split='train')

In [ ]:
sample = dataset[0]
display(sample['image'].resize((256, 256)))
print(sample['text'])

In [ ]:
notebook_login()

In [ ]:
ckpt_path = hf_hub_download(repo_id="CompVis/stable-diffusion-v-1-4-original", filename="sd-v1-4-full-ema.ckpt", use_auth_token=True)

In [ ]:
BATCH_SIZE = 8 # fixed # (1epoch) = (data size) / (batch size)
NUM_GPUS = 1 # fixed
ACCUMULATE_BATCHES = 1

gpu_list = ",".join((str(x) for x in range(NUM_GPUS))) + ","
print(f"Using GPUs: {gpu_list}")

In [ ]:
# make & fix configs/stable-diffusion/fashion.yaml

In [ ]:
!python main.py \
    --train \
    --base configs/stable-diffusion/fashion.yaml \
    --gpus "$gpu_list" \
    --scale_lr False \
    --num_nodes 1 \
    --check_val_every_n_epoch 10 \
    --finetune_from "$ckpt_path" \
    data.params.batch_size="$BATCH_SIZE" \
    lightning.trainer.accumulate_grad_batches="$ACCUMULATE_BATCHES" \
    data.params.validation.params.n_gpus="$NUM_GPUS"

In [ ]:
#!python scripts/txt2img.py \
#    --prompt '' \
#    --outdir 'outputs/generated_fashion' \
#    --H 512 --W 512 \
#    --n_samples 4 \
#    --config 'configs/stable-diffusion/fashion.yaml' \
#    --ckpt 'path/to/your/checkpoint'

In [ ]:
#prompt_img = Image.open("outputs/generated_fashion/grid-0000.png").resize((1024, 256))
#display(prompt_img)